# Open-Meteo : Weather Forecast API

Open-Meteo features weather forecasts aswell as historical weather data, with up to 1km resolution.

We'll use the free available licence for non-commercial use, which restricts its usage to less than 10 000 daily requests.

Forecast documentation : https://open-meteo.com/en/docs

Historical Data Documentation : https://open-meteo.com/en/docs/historical-weather-api

# Imports

In [74]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

from geopy.geocoders import Nominatim
import numpy as np

# Documentation

Weather Variables : WMO Weather interpretation codes (WW)

In [75]:
wmo_codes = pd.DataFrame({"code": [0, 1, 2, 3, 45, 48, 51, 53, 55, 56, 57, 61, 63, 65, 66, 67, 71, 73, 75, 77, 80, 81, 82, 85, 86, 95, 96, 99]})
wmo_codes['description'] = np.select(
    [wmo_codes['code'].isin([0]),
     wmo_codes['code'].isin([1, 2, 3]),
     wmo_codes['code'].isin([45, 48]),
     wmo_codes['code'].isin([51, 53, 55]),
     wmo_codes['code'].isin([56, 57]),
     wmo_codes['code'].isin([61, 63, 65]),
     wmo_codes['code'].isin([66, 67]),
     wmo_codes['code'].isin([71, 73, 75]),
     wmo_codes['code'].isin([77]),
     wmo_codes['code'].isin([80, 81, 82]),
     wmo_codes['code'].isin([85, 86]),
     wmo_codes['code'].isin([95]),
     wmo_codes['code'].isin([96, 99]),
    ],
    ['Clear sky',
     'Mainly clear, partly cloudy, and overcast',
     'Fog and depositing rime fog',
     'Drizzle: Light, moderate, and dense intensity',
     'Freezing Drizzle: Light and dense intensity',
     'Rain: Slight, moderate and heavy intensity',
     'Freezing Rain: Light and heavy intensity',
     'Snow fall: Slight, moderate, and heavy intensity',
     'Snow grains',
     'Rain showers: Slight, moderate, and violent',
     'Snow showers slight and heavy',
     'Thunderstorm: Slight or moderate',
     'Thunderstorm with slight and heavy hail'
    ]
)
print("WMO weather codes :")
display(wmo_codes)

WMO weather codes :


,code,description
0,0,Clear sky
1,1,"Mainly clear, partly cloudy, and overcast"
2,2,"Mainly clear, partly cloudy, and overcast"
3,3,"Mainly clear, partly cloudy, and overcast"
4,45,Fog and depositing rime fog
5,48,Fog and depositing rime fog
6,51,"Drizzle: Light, moderate, and dense intensity"
7,53,"Drizzle: Light, moderate, and dense intensity"
8,55,"Drizzle: Light, moderate, and dense intensity"
9,56,Freezing Drizzle: Light and dense intensity


# Simple requests

## Documentation's example usecase

Getting the weather forecasts at ESME's Ivry-Sur-Seine campus.

In [76]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 48.81452162820077,
	"longitude": 2.3948078406657114,
	"hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "weather_code"],
    "timezone": "auto"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]

Quick check of the selected address

In [77]:
geolocator = Nominatim(user_agent="ESMEAirPollutionPrediction")
location = geolocator.reverse(f"{params['latitude']}, {params['longitude']}")
print(location.address)

60, Boulevard de Brandebourg, Ivry Port, Ivry-sur-Seine, L'Haÿ-les-Roses, Val-de-Marne, Île-de-France, France métropolitaine, 94200, France


In [78]:
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()

hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit = "s"),
        end=pd.to_datetime(hourly.TimeEnd(), unit = "s"),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
        )
    }
for i in range(hourly.VariablesLength()):  # Iterating through all the variables in params["hourly"]
    hourly_data[params["hourly"][i]] = hourly.Variables(i).ValuesAsNumpy()

hourly_dataframe = pd.DataFrame(data=hourly_data)
display(hourly_dataframe)

Coordinates 48.81999969482422°E 2.3999996185302734°N
Elevation 36.0 m asl
Timezone b'Europe/Paris' b'CET'
Timezone difference to GMT+0 3600 s


,date,temperature_2m,relative_humidity_2m,precipitation,weather_code
0,2023-11-04 23:00:00,11.456500,82.0,0.1,61.0
1,2023-11-05 00:00:00,11.406500,85.0,0.3,61.0
2,2023-11-05 01:00:00,11.156500,84.0,0.8,61.0
3,2023-11-05 02:00:00,11.106501,82.0,1.4,61.0
4,2023-11-05 03:00:00,11.106501,81.0,0.0,80.0
...,...,...,...,...,...
163,2023-11-11 18:00:00,8.949500,85.0,0.0,3.0
164,2023-11-11 19:00:00,8.299500,88.0,0.0,3.0
165,2023-11-11 20:00:00,7.749500,90.0,0.0,3.0
166,2023-11-11 21:00:00,7.249500,90.0,0.0,3.0


matching the weather codes to the wmo df :

In [80]:
if not "description" in hourly_dataframe.columns: 
    hourly_dataframe = hourly_dataframe.merge(wmo_codes, left_on="weather_code", right_on="code")
print("weather forecast at", location.address)
display(hourly_dataframe)

weather forecast at 60, Boulevard de Brandebourg, Ivry Port, Ivry-sur-Seine, L'Haÿ-les-Roses, Val-de-Marne, Île-de-France, France métropolitaine, 94200, France


,date,temperature_2m,relative_humidity_2m,precipitation,weather_code,code,description
0,2023-11-04 23:00:00,11.456500,82.0,0.1,61.0,61,"Rain: Slight, moderate and heavy intensity"
1,2023-11-05 00:00:00,11.406500,85.0,0.3,61.0,61,"Rain: Slight, moderate and heavy intensity"
2,2023-11-05 01:00:00,11.156500,84.0,0.8,61.0,61,"Rain: Slight, moderate and heavy intensity"
3,2023-11-05 02:00:00,11.106501,82.0,1.4,61.0,61,"Rain: Slight, moderate and heavy intensity"
4,2023-11-05 04:00:00,10.956500,81.0,0.1,61.0,61,"Rain: Slight, moderate and heavy intensity"
...,...,...,...,...,...,...,...
163,2023-11-10 19:00:00,8.549500,83.0,0.0,1.0,1,"Mainly clear, partly cloudy, and overcast"
164,2023-11-10 20:00:00,7.899500,87.0,0.0,1.0,1,"Mainly clear, partly cloudy, and overcast"
165,2023-11-10 21:00:00,7.349500,90.0,0.0,1.0,1,"Mainly clear, partly cloudy, and overcast"
166,2023-11-07 18:00:00,8.886500,82.0,0.0,0.0,0,Clear sky
